<a href="https://colab.research.google.com/github/G1-ABID-23-24/offensive-language-detection-2024/blob/main/2_twitter_roberta_base_offensive_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('./train.csv')

ParserError: Error tokenizing data. C error: EOF inside string starting at row 6152

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [ ]:
en_stopwords = nlp.Defaults.stop_words

def remove_stop_words(text):
    cleanText = ''
    phrase = nlp(text)
    for token in phrase:
        if not token.is_stop and not token.is_punct and not token.like_url:
            cleanText += ' ' + token.text

    return cleanText

df['text_cleaned'] = df['text'].apply(remove_stop_words)

In [ ]:
df['text_cleaned']

Separamos el conjunto en entrenamiento y test.

In [ ]:
# spling training and testing
from sklearn.model_selection import train_test_split

X = df['text_cleaned']
y = df['label']

#train
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.33, random_state=42)

NameError: name 'df' is not defined

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='offensive'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

encoded_input = tokenizer(df['text_cleaned'], return_tensors='pt')
# output = model(**encoded_input)

predictions = model.predict(encoded_input)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(df['label'], predictions)